## Chain of thoughts prompting

In [1]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import OpenAI,ChatOpenAI

from langchain_core.prompts import PromptTemplate
# Load environment variables from .env
load_dotenv()


True

In [2]:
# Create a ChatOpenAI model
model = ChatOpenAI(model="gpt-4o")

Chain of thought prompt — a series of intermediate reasoning steps — helps the model to provide the correct answer.

The reasoning  steps /breakdown that are provide in chain-of-thought exemplars is similar to the natural language internal monologue a person has as he / she think through a problem or task.

* Model to think through the problem in a step-by-step manner
* Generate the intermediate steps without jumping straight to the final answer
* The reasoning steps in the response also provide interpretability of how the LLM arrived at the final output.

Strategies for COT prompt : 
 * Explicit instructions

In [3]:
input_text = """
¡Preparar café Cold Brew es un proceso sencillo y refrescante!
Todo lo que necesitas son granos de café molido grueso y agua fría.
Comienza añadiendo el café molido a un recipiente o jarra grande.
Luego, vierte agua fría, asegurándote de que todos los granos de café
estén completamente sumergidos.
Remueve la mezcla suavemente para garantizar una saturación uniforme.
Cubre el recipiente y déjalo en remojo en el refrigerador durante al
menos 12 a 24 horas, dependiendo de la fuerza deseada.
"""

In [4]:

prompt_template = PromptTemplate.from_template("""Give me a numbered list of all coffee-related words in English from the text below:
Text: {input_text}""")

prompt=prompt_template.invoke({"input_text": input_text})

In [5]:
result = model.invoke(prompt)
print("\n\n")
print(result.content)




1. café
2. Cold Brew
3. granos
4. café molido
5. café
6. granos de café


In [9]:
prompt_template = PromptTemplate.from_template("""Give me a numbered list of all coffee-related words in English from the text below:
The task requires the following actions:
1 - Translate the given text into English.
2 - Extract only the coffee-related Ingredients
3 - Exclude generic liquids or additives (milk, sugar, water, syrup) 
4 - Exclude utensils or actions (boil, stir, mix, pour)
5 - List each coffee-related word from the English text.
Text: {input_text}""")

prompt=prompt_template.invoke({"input_text": input_text})

In [10]:
prompt_template = PromptTemplate.from_template("""
You are an expert linguist and food-domain text analyst.
Your task is to find **coffee-related words** in the given text, following a deliberate reasoning process.
---
### 🔍 Step-by-Step Reasoning
1 - Extract only the coffee type or ingredient.  
2 - Exclude generic liquids or additives or utensils. 
3 - Translate the given text into English.  
4 - List each coffee-related word from the English text as a numbered list.
---

### 🧾 Input Text
{input_text}
---
### ✅ Expected Output Format
1. Coffee  
2. Arabica 
---
""")

prompt=prompt_template.invoke({"input_text": input_text})

In [11]:
result = model.invoke(prompt)
print("\n\n")
print(result.content)




### 🔍 Step-by-Step Reasoning

1. **Extract only the coffee type or ingredient.**  
   - The text mentions "café Cold Brew", "granos de café molido grueso", and "café molido". Here, "café" refers to coffee as an ingredient.

2. **Exclude generic liquids or additives or utensils.**  
   - Excluded items would include "agua fría" (cold water) as it is a generic liquid and not specific to coffee.

3. **Translate the given text into English.**  
   - The translation is: "Preparing Cold Brew coffee is a simple and refreshing process! All you need are coarsely ground coffee beans and cold water. Start by adding the ground coffee to a large container or jug. Then, pour cold water, ensuring that all the coffee grounds are completely submerged. Stir the mixture gently to ensure even saturation. Cover the container and let it steep in the refrigerator for at least 12 to 24 hours, depending on the desired strength."

4. **List each coffee-related word from the English text as a numbered list.**

* Implicit instructions

In [12]:
# Correct answer: 360, 375.
question = """A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.
Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.
Tomorrow they reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.
How many units can they produce today? How many tomorrow?"""

one_shot_exemplar = """Q: Roger has 5 tennis balls.
He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
Q: """

result = model.invoke(f"{question}\nA:") 
#result = chat.invoke(f"{one_shot_exemplar}{question}\nA:")
#print("Full result:")
#print(result)
print("Content only:")
print(result.content)

Content only:
To calculate the number of units Megacorp can produce today, we need to consider the number of factories they own and their respective efficiencies.

**Today:**
- High efficiency factories: 3 factories × 100 units/factory = 300 units
- Low efficiency factories: 2 factories × 30 units/factory = 60 units

Total units produced today = 300 (high) + 60 (low) = 360 units

**Tomorrow:**
- High efficiency factories: 3 factories × 100 units/factory = 300 units
- Medium efficiency factory: 1 factory × 60 units/factory = 60 units
- Low efficiency factory with outage: 1 factory × 30 units/factory × 0.5 = 15 units

Total units produced tomorrow = 300 (high) + 60 (medium) + 15 (low with outage) = 375 units

Therefore, Megacorp can produce 360 units today and 375 units tomorrow.


In [13]:
# Create a  model

model = ChatOpenAI(
    model="gpt-4o" , model_kwargs={"max_tokens": 1000, "temperature": 0.7}
)

C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'max_tokens', 'temperature'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [14]:
context = """Given a JSON entry of a data source, output a JSON with the following fields and explain the reasoning:
pii: True/False, the dataset contains Personally Identifiable Information.
age: How many years since the dataset was last modified.
keywords: New keywords to index this dataset under, beyond the current set of keywords.
The last text output should be the JSON.
"""


question = """
{
    "@type" : "dcat:Dataset",
    "description" : "The MDS 3.0 Frequency Report summarizes information for active residents currently in nursing homes. The source of these counts is the residents MDS assessment record. The MDS assessment information for each active nursing home resident is consolidated to create a profile of the most recent standard information for the resident.\n",
    "title" : "MDS 3.0 Frequency Report",
    "accessLevel" : "public",
    "identifier" : "465",
    "license" : "http://opendefinition.org/licenses/odc-odbl/",
    "modified" : "2016-04-05",
    "temporal" : "2012-01-01T00:00:00-05:00/2015-12-31T00:00:00-05:00",
    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:HealthData@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "Activities of Daily Living (ADL)" ],
    "language" : [ "en" ],
    "programCode" : [ "009:000" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }


"""

result = model.invoke(f"{context}\nJSON:{question}\nAnswer:")
#print("Full result:")
#print(result)
print("Content only:")
print(result.content)



Content only:
To determine the values for the specified fields, let's analyze the provided JSON data:

1. **pii (Personally Identifiable Information)**:
   - The description mentions "MDS assessment information for each active nursing home resident," which implies that the dataset could potentially include personal information about residents in nursing homes.
   - However, the JSON entry does not explicitly list any fields such as names, social security numbers, or other direct identifiers. The presence of "consolidated" and "profile" might suggest aggregation, but without explicit identifiers, it's tricky.
   - Given the context, we can infer that the dataset may contain PII because it deals with individual assessments related to health data, even though it's not explicitly listed in the JSON.

2. **age**:
   - The "modified" field indicates the dataset was last updated on "2016-04-05."
   - Assuming the current year is 2023, the age of the dataset is calculated as 2023 - 2016 = 7 ye

In [15]:
one_shot_exemplar = """
{

    "@type" : "dcat:Dataset",
    "description" : "The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries.",
    "title" : "Medicare Multi-Carrier Claims System",
    "accessLevel" : "restricted public",
    "dataQuality" : true,
    "identifier" : "b6ffafab-1cfd-42dd-b8cb-7a554efaefa7",
    "landingPage" : "http://www.cms.gov/Research-Statistics-Data-and-Systems/Computer-Data-and-Systems/Privacy/Systems-of-Records-Items/09-70-0501-MCS.html",
    "license" : "http://www.usa.gov/publicdomain/label/1.0/",
    "modified" : "2014-09-30",
    "rights" : "Contains personally identifiable information and is subject to the Privacy Act of 1974, as amended at 5 United States Code (U.S.C.) 552a.  Requests should be directed to the appropriate System Manager, identified in the System of Records notice.",
    "primaryITInvestmentUII" : "009-000004256, 009-000004254",
    "systemOfRecords" : "09-70-0501",

    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:Healthdata@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "medicare", "part b", "claims" ],
    "programCode" : [ "009:078" ],
    "theme" : [ "Medicare" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }

Answer: The 'rights' tag says 'Contains personally identifiable information' so pii is True.
The 'modified' tag is '2014-09-30'. The current year is 2024, 2024 minus 2014 is 10, so the age is 10.
To determine keywords I will look at all the fields that describe the dataset.
Then I will take the most salient and distinctive aspects of the fields and make those keywords.
Looking at all the fields, the ones that describe the dataset are  "description" and "title".
The "title" field is "Medicare Multi-Carrier Claims System".
Good keywords from the "title" field are "medicare" and "claims".
The "description" field is ""The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries."
Good keywords from the "description" field are "medical insurance benefits".
Good proposed keywords from both fields are "medicare", "claims", and "medical insurance benefits".
Next inspect the "keyword" field to make sure the proposed keywords are not already included.
The "keyword" field contains the keywords "medicare", "part b", and "claims".
From our proposed keywords, "medicare" should not be output since it is already in the "keyword" field.
That leaves "claims" and "medical insurance benefits" as proposed keywords.

JSON:
{
  "pii" : true,
  "age" : 10,
  "keywords" : ["claims", "medical insurance benefits"]
}
"""

result =  model.invoke(f"{context}{one_shot_exemplar}\nJSON:{question}\nAnswer:")
#print("Full result:")
#print(result)
print("Content only:")
print(result.content)

Content only:
To generate the required JSON fields (`pii`, `age`, `keywords`) based on the provided dataset, let's break down the information:

1. **PII (Personally Identifiable Information):**
   - The JSON entry does not explicitly state that it contains personally identifiable information, unlike the previous example where it was clearly mentioned under the "rights" field. Therefore, we will assume `pii` is `False`.

2. **Age:**
   - The "modified" date of the dataset is "2016-04-05".
   - Assuming the current year is 2024 (as inferred from the previous explanation), the age of the dataset is calculated as: 
     \[
     2024 - 2016 = 8
     \]
   - Therefore, the dataset is 8 years old.

3. **Keywords:**
   - Current keyword is "Activities of Daily Living (ADL)".
   - From the "title" field: "MDS 3.0 Frequency Report", potential keywords are "MDS 3.0" and "Frequency Report".
   - From the "description" field: "The MDS 3.0 Frequency Report summarizes information for active residents